In [41]:
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from berny import Berny, geomlib, coords,Geometry,Math
from FcMole import FcM,FcM_like
from AP_class import APDFT_perturbator
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np 
from numpy import dot,log
from numpy.linalg import norm as norm
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
from pyscf.geomopt.berny_solver import to_berny_geom as tbg
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109
np.set_printoptions(precision=3)

In [2]:
import ipyvolume as ipv
from mpl_toolkits.mplot3d import Axes3D

In [25]:
def anglecenter(coords):
    return np.around(np.arccos(coords[1].dot(coords[2])/norm(coords[1])/norm(coords[2]))*180/np.pi,decimals=3)

In [3]:
l=3.3376
x=-(1/24)**.5*l
z=(2/3)**.5*l+x
ch4str="C 0 0 0; H 0 0  {}; H  0  {} {}; H {} {} {}; H {} {} {}".format(z,l/3**.5,x,l/2,-l*3**.5/6,x,-l/2,-l*3**.5/6,x)

In [4]:
CH4=gto.M(atom=ch4str,unit="bohr",basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})
mf = scf.RHF(CH4)
mf.scf()

converged SCF energy = -40.2151804034983


-40.215180403498316

In [5]:
g0=mf.Gradients().grad()

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000    -0.0000000000
1 H     0.0000000000     0.0000000000    -0.0000007926
2 H     0.0000000000    -0.0000007473     0.0000002642
3 H    -0.0000006472     0.0000003736     0.0000002642
4 H     0.0000006472     0.0000003736     0.0000002642
----------------------------------------------


In [6]:
h0=mf.Hessian().hess()

In [7]:
ch4coords=CH4.atom_coords()

In [8]:
afC=aaff(mf,[1,0,0,0,0])+alc_deriv_grad_nuc(CH4,[1,0,0,0,0])
afH1=aaff(mf,[0,0,0,0,-1])+alc_deriv_grad_nuc(CH4,[0,0,0,0,-1])
afH2=aaff(mf,[0,0,0,-1,0])+alc_deriv_grad_nuc(CH4,[0,0,0,-1,0])
afH3=aaff(mf,[0,0,-1,0,0])+alc_deriv_grad_nuc(CH4,[0,0,-1,0,0])

In [9]:
afH1

array([[-0.02541652, -0.01467424, -0.01037625],
       [-0.01250345, -0.00721887, -0.01100032],
       [-0.01250345, -0.01277749, -0.00313924],
       [-0.01731736, -0.00443956, -0.00313924],
       [ 0.06774078,  0.03911016,  0.02765506]])

In [10]:
for i in (afH1+afC):
    print(norm(i))

0.03112875636394686
0.06943666146071746
0.06943666146069727
0.06943666146071918
0.0040457755480539925


In [11]:
for i in range(5):
    print(norm(h0[i,i]))

1.0345685841811765
0.37597489317648214
0.375974893176569
0.37597489317647736
0.37597489317651916


In [12]:
.37597489317647576*.01815090704294456

0.006824285336527222

In [13]:
afH1

array([[-0.02541652, -0.01467424, -0.01037625],
       [-0.01250345, -0.00721887, -0.01100032],
       [-0.01250345, -0.01277749, -0.00313924],
       [-0.01731736, -0.00443956, -0.00313924],
       [ 0.06774078,  0.03911016,  0.02765506]])

In [14]:
for i in afH1:
    print(norm(i))

0.031128756363930567
0.018150907042928623
0.018150907042942047
0.018150907042941075
0.0829651766432179


In [80]:
bch4=Berny(tbg(CH4))
s=bch4._state
vars(s)

{'geom': <Geometry 'CH4'>,
 'params': {'gradientmax': 0.00045,
  'gradientrms': 0.00015,
  'stepmax': 0.0018,
  'steprms': 0.0012,
  'trust': 0.3,
  'dihedral': True,
  'superweakdih': False},
 'trust': 0.3,
 'coords': <InternalCoords "bonds: 4, angles: 6, dihedrals: 0">,
 'H': array([[0.478, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   ],
        [0.   , 0.478, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   ],
        [0.   , 0.   , 0.478, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   ],
        [0.   , 0.   , 0.   , 0.478, 0.   , 0.   , 0.   , 0.   , 0.   ,
         0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.169, 0.   , 0.   , 0.   , 0.   ,
         0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   , 0.169, 0.   , 0.   , 0.   ,
         0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.169, 0.   , 0.   ,
         0.   ],
        [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.169, 0.   ,
         0.   ],
        [0

In [81]:
Q=s.future.q

In [82]:
#come trasforma g in coordinate interne???
B = s.coords.B_matrix(s.geom)
B_inv = B.T.dot(Math.pinv(np.dot(B, B.T), log=log))
g_ic=dot(B_inv.T, (afH1+afC).reshape(-1))

In [83]:
g_ic

array([ 0.068,  0.068,  0.068, -0.004,  0.015,  0.015, -0.015,  0.015,
       -0.015, -0.015])

In [84]:
h_ic.diagonal()

array([0.369, 0.369, 0.369, 0.369, 0.12 , 0.12 , 0.12 , 0.12 , 0.12 ,
       0.12 ])

In [85]:
-g_ic/h_ic.diagonal()

array([-0.184, -0.184, -0.184,  0.011, -0.123, -0.123,  0.123, -0.123,
        0.123,  0.123])

In [86]:
s.H=h_ic

In [87]:
bch4.send((1.,afH1+afC))

In [88]:
bch4._state.predicted.q-bch4._state.previous.q

array([-0.149, -0.149, -0.149,  0.004, -0.062, -0.062,  0.062, -0.062,
        0.062,  0.062])

In [78]:
bch4._state.predicted.q-bch4._state.previous.q

array([-0.149, -0.149, -0.149,  0.004, -0.062, -0.062,  0.062, -0.062,
        0.062,  0.062])

In [77]:
1.848*180/np.pi

105.88260054017614

In [43]:
B

array([[ 0.   ,  0.   , -1.   ,  0.   ,  0.   ,  1.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   , -0.943,  0.333,  0.   ,  0.   ,  0.   ,  0.   ,  0.943,
        -0.333,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [-0.816,  0.471,  0.333,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.816, -0.471, -0.333,  0.   ,  0.   ,  0.   ],
       [ 0.816,  0.471,  0.333,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   , -0.816, -0.471, -0.333],
       [ 0.   ,  0.652,  0.461,  0.   , -0.489,  0.   ,  0.   , -0.163,
        -0.461,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.565, -0.326,  0.461, -0.424,  0.245,  0.   ,  0.   ,  0.   ,
         0.   , -0.141,  0.082, -0.461,  0.   ,  0.   ,  0.   ],
       [-0.565, -0.326,  0.461,  0.424,  0.245,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.141,  0.082, -0.461],
       [ 0.565,  0.326, -0.461,  0.   ,  

In [48]:
#come posso trasformare H in coordinate interne??
h_ic=dot(B_inv.T,(np.swapaxes(h0,1,2)).reshape(15,15)).dot(B_inv)

In [27]:
anglecenter(CH4.atom_coords())/180*np.pi

1.9106293854507124

In [51]:
Q

array([2.044, 2.044, 2.044, 2.044, 1.911, 1.911, 1.911, 1.911, 1.911,
       1.911])

In [57]:
np.linalg.inv(h_ic).dot(g_ic)

array([-0.031, -0.024, -0.031, -0.21 , -0.125, -0.188, -0.25 ,  0.   ,
       -0.25 , -0.312])

In [94]:
h_ic.diagonal()

array([0.369, 0.369, 0.369, 0.369, 0.12 , 0.12 , 0.12 , 0.12 , 0.12 ,
       0.12 ])

In [93]:
np.linalg.eig((np.swapaxes(h0,1,2)).reshape(15,15))

(array([ 3.784e-01,  8.944e-01,  8.944e-01,  8.944e-01,  1.359e-01,
         1.359e-01,  1.359e-01,  1.060e-01,  1.060e-01,  4.535e-12,
        -1.041e-11, -4.951e-11, -4.021e-07, -4.021e-07, -4.021e-07]),
 array([[-7.332e-13,  9.353e-03,  8.024e-01, -1.582e-02, -3.751e-01,
         -1.117e-01,  5.184e-02,  5.053e-11,  3.906e-11, -1.791e-01,
          4.078e-01,  3.997e-02,  5.510e-06, -2.280e-05,  6.131e-06],
        [-1.104e-12, -8.021e-01,  8.825e-03, -2.684e-02, -6.551e-02,
          4.027e-02, -3.872e-01, -5.341e-11, -4.136e-11, -4.090e-01,
         -1.805e-01,  9.814e-03,  9.999e-06,  1.013e-05,  1.115e-05],
        [-4.894e-12,  2.665e-02, -1.611e-02, -8.020e-01, -1.043e-01,
          3.765e-01,  5.680e-02,  9.705e-13,  1.013e-12, -2.508e-02,
          3.263e-02, -4.453e-01, -2.736e-06,  2.176e-06, -3.053e-06],
        [ 2.379e-12, -5.197e-04, -4.458e-02,  8.788e-04,  3.952e-01,
          1.177e-01, -5.461e-02,  2.969e-01, -3.915e-01, -1.791e-01,
          4.079e-01,  3.997e-02,

In [33]:
nh3str="""N  -0.000000  -0.128885  -0.043079;   H   0.000000   0.067780   0.935168;   
   H   0.808204   0.295141  -0.446045;  
   H  -0.808204   0.295141  -0.446045;
   """

In [34]:
#NH3=gto.M(atom="N 0 0 0;H 0 0 1.8; H 0.9 0.5 -.9; H -0.9 0.5 -.9",unit="bohr",
NH3=gto.M(atom=nh3str,
            basis={"H":"pc-2",'N':bse.get_basis("pcX-2",fmt="nwchem",elements=[7])})
mfnh3 = scf.RHF(NH3)
mfnh3.scf()

converged SCF energy = -56.2221835947891


-56.222183594789115

In [35]:
nh3_eq=optimize(mfnh3)


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N   0.000000  -0.128885  -0.043079    0.000000  0.000000  0.000000
   H   0.000000   0.067780   0.935168    0.000000  0.000000  0.000000
   H   0.808204   0.295141  -0.446045    0.000000  0.000000  0.000000
   H  -0.808204   0.295141  -0.446045    0.000000  0.000000  0.000000
converged SCF energy = -56.2221835947893
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 N     0.0000000000     0.0000180200    -0.0001418131
1 H     0.0000000000     0.0000355141     0.0000167946
2 H    -0.0000802407    -0.0000267671     0.0000625092
3 H     0.0000802407    -0.0000267671     0.0000625092
----------------------------------------------
cycle 1: E = -56.2221835948  dE = -56.2222  norm(grad) = 0.000210009


In [36]:
nh3optg=nh3_eq.atom_coords()

In [37]:
norm(nh3optg[1]-nh3optg[0])

1.8856061076490165

In [42]:
OH2=gto.M(atom="O 0 0 0;H -0.20216112 -0.1617289 1.75663003; H 1.37299943  1.09839954 -0.24739346",unit="bohr",
            basis={"H":"pc-2",'O':bse.get_basis("pcX-2",fmt="nwchem",elements=[8])})
mfoh2 = scf.RHF(OH2)
mfoh2.scf()

converged SCF energy = -76.0633193327809


-76.06331933278094

In [43]:
oh2_eq=optimize(mfoh2)


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H  -0.106979  -0.085583   0.929569    0.000000  0.000000  0.000000
   H   0.726560   0.581248  -0.130915    0.000000  0.000000  0.000000
converged SCF energy = -76.063319332781
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 O    -0.0000165554    -0.0000132444    -0.0000136389
1 H     0.0000057728     0.0000046183     0.0000071441
2 H     0.0000107826     0.0000086261     0.0000064949
----------------------------------------------
cycle 1: E = -76.0633193328  dE = -76.0633  norm(grad) = 3.121e-05


In [41]:
oh2optg=oh2_eq.atom_coords()
oh2optg-oh2optg[0]

array([[ 0.        ,  0.        ,  0.        ],
       [-0.20216112, -0.1617289 ,  1.75663003],
       [ 1.37299943,  1.09839954, -0.24739346]])

In [48]:
FH=gto.M(atom="F 0 0 0;H 0 0 1.69574139",unit="bohr",
            basis={"H":"pc-2",'F':bse.get_basis("pcX-2",fmt="nwchem",elements=[9])})
mffh = scf.RHF(FH)
mffh.scf()
fh_eq=optimize(mffh)

converged SCF energy = -100.065540168732

Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   F   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   H   0.000000   0.000000   0.897348    0.000000  0.000000  0.000000
converged SCF energy = -100.065540168732
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 F     0.0000000000    -0.0000000000    -0.0000012767
1 H    -0.0000000000     0.0000000000     0.0000012767
----------------------------------------------
cycle 1: E = -100.065540169  dE = -100.066  norm(grad) = 1.80549e-06


In [49]:
fhoptg=fh_eq.atom_coords()
fhoptg-fhoptg[0]

array([[0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.69574139]])

In [50]:
#ch4coords,h0,afC,afH1,nh3optg,oh2optg,fhoptg

In [51]:
#getting inverse of the hessian matrix
H1=np.linalg.inv((np.swapaxes(h0,1,2)).reshape(15,15))
h1=H1.reshape(5,3,5,3)

In [52]:
nh3predictg=(ch4coords-np.einsum('ijkl,kl->ij',h1,  afC+afH1))[:4]
oh2predictg=(ch4coords-np.einsum('ijkl,kl->ij',h1,  2*afC+afH1+afH2))[:3]
fhpredictg=(ch4coords-np.einsum('ijkl,kl->ij',h1,  3*afC+afH1+afH2+afH3))[:2]

In [53]:
#place non H atom in the origin
nh3predictg-=nh3predictg[0]
oh2predictg-=oh2predictg[0]
fhpredictg-=fhpredictg[0]
nh3optg-=nh3optg[0]
oh2optg-=oh2optg[0]
fhoptg-=fhoptg[0]

In [54]:
#bond lengths
print(norm(ch4coords[1]))
print(norm(nh3predictg[1]),norm(nh3optg[1]))
print(norm(oh2predictg[1]),norm(oh2optg[1]))
print(norm(fhpredictg[1]),norm(fhoptg[1]))

2.043854241378284
1.8771027614531632 1.8856061076490165
1.7017431434979045 1.7756053637844758
1.514804488221807 1.69574139


In [56]:
#angles
print(anglecenter(ch4coords))
print(anglecenter(nh3predictg),anglecenter(nh3optg))
print(anglecenter(oh2predictg),anglecenter(oh2optg))

109.471
103.102 108.193
96.677 106.393


In [58]:
ipv.figure()
ipv.scatter(ch4coords[:,0],ch4coords[:,1],ch4coords[:,2],color="black",marker='sphere',size=12)
ipv.scatter(nh3predictg[:,0],nh3predictg[:,1],nh3predictg[:,2],color="blue",marker='sphere',size=12)
ipv.scatter(oh2predictg[:,0],oh2predictg[:,1],oh2predictg[:,2],color="green",marker='sphere',size=12)
ipv.scatter(fhpredictg[:,0],fhpredictg[:,1],fhpredictg[:,2],color="yellow",marker='sphere',size=12)
ipv.scatter(np.asarray([0.]),np.asarray([0.]),np.asarray([0.]),color="red",marker='sphere',size=12)
ipv.show()

In [166]:
ipv.figure()
ipv.scatter(ch4coords[:,0],ch4coords[:,1],ch4coords[:,2],color="black",marker='sphere',size=12)
ipv.scatter(nh3optg[:,0],nh3optg[:,1],nh3optg[:,2],color="blue",marker='sphere',size=12)
ipv.scatter(oh2optg[:,0],oh2optg[:,1],oh2optg[:,2],color="green",marker='sphere',size=12)
ipv.scatter(fhoptg[:,0],fhoptg[:,1],fhoptg[:,2],color="yellow",marker='sphere',size=12)
ipv.show()